In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dash
from dash import dcc, html
import networkx as nx
import dash_cytoscape as cyto
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import matplotlib.cm as cm #or 
import matplotlib as mpl
import plotly.express as px
import io
import base64
import math as ma
#pip install dash
#pip install dash-cytoscape
#pip install dash dash-cytoscape
df = pd.read_csv('MIP_19.csv') #leemos la base

In [2]:
#Funciones generales
def crear_red(df2):# función para crear la red apartir de la matriz de adyacencia   
    # Crear un grafo dirigido vacío
    G = nx.DiGraph()

    # Agregar nodos al grafo
    for nodo in df2.columns:
        if nodo!=0:
            G.add_node(nodo)

    # Agregar arcos ponderados (con peso) al grafo
    for origen in df2.index:
        peso_arco_max=0
        for destino in df2.columns: #las flechas van de renglón a columna
            if destino!=0:
                peso = df2.loc[origen, destino]
                if peso != 0: #no se cuentan los pesos de valor 0
                    G.add_edge(origen, destino, weight=peso)
                if peso_arco_max<=df2[destino].max():
                    peso_arco_max=df2[destino].max()
    #Mostramos el grafo
    #print("Nodos:", G.nodes())
    #print("Num. nodos:", len(G.nodes()))
    #print("Num. de arcos:", len(G.edges))
    #print("Peso máximo de arco:", peso_arco_max)
    return G, peso_arco_max

def cmap_at_scale(cmap, bins):
    # cmap:  mpl or cmocean colormap 
    # bins :  ascending ordered list  of floats in [0,1] that defines the scale for a plotly colorscale derived from cmap
    # returns   a list of hexcolors
    
    if not isinstance(bins, (list, np.ndarray)):
        raise ValueError('bins should be a 1d list or  an array')
    if not  (0 <= np.asarray(bins).min() <= 1) or  not  (0 <= np.asarray(bins).max() <= 1):
        raise ValueError('The elements in bins should be in [0,1]')
    scale  = sorted(bins)
    colors = cmap(scale)[:, :3]
    colors = (255*colors+0.5).astype(np.uint8)
    hexcolors = [f'#{c[0]:02x}{c[1]:02x}{c[2]:02x}' for c in colors]
    return hexcolors

def val_to_color(val, bins, colors):
    # val: float in [0,1]
    # colors:  a list of hex colors representing the Plotly colorscale with scale bins and colors colors
    if not isinstance(colors[0], str):
        raise ValueError('The elements of colors must be strings (hex colors)')
    if colors[0][0] != '#':
        raise ValueError('Your color codes are not hex codes')
    if len(bins) != len(colors):
        raise ValueError('bins and colors must have the same length') 
    
    # find the index of the  bin  that gives the interval the val belongs to
    # see https://docs.scipy.org/doc/numpy/reference/generated/numpy.digitize.html
    idx = np.digitize(val, bins)

    # associate to val a color in colors
    vcolor = colors[idx-1]
    
    return   idx, vcolor 

def arreglo_ejemplo(G, d=2):
    lista=[]
    for node in G.nodes():
        lista.append(int(str(node)[:d]))# Quedarse con los "d" primeros dígitos únicos de todos los nodos 
    return lista

def definir_nuevo_atributo_nodo(G, arreglo):
    dic={}        
    i=0
    if len(G.nodes())==len(arreglo): #condición de consistencia
        for node in G.nodes():
            dic[node]=arreglo[i]
            i+=1
        if len(str(arreglo[0]))==2: #caso para sector
            for node in dic.keys():
                G.nodes[node]["sector"]=dic[node]
        elif len(str(arreglo[0]))==3: #caso para subsector
            for node in dic.keys():
                G.nodes[node]["subsector"]=dic[node]
        elif len(str(arreglo[0]))==4: #caso para rama
            for node in dic.keys():
                G.nodes[node]["rama"]=dic[node]
    return G, dic

def contraer_red(G, df2, arreglo):
    newG_y_dic=definir_nuevo_atributo_nodo(G, arreglo)
    G=newG_y_dic[0] #redefinimos la red
    unique_nodes=list(set(newG_y_dic[1].values()))
    unique_nodes=sorted(unique_nodes)
    d=len(unique_nodes)
    #print(df2) #print(d)
    i, j=0, 0 #contadores
    array_nodes_contract=[]
    if len(str(arreglo[0]))==2: #caso para sector
        for unique_node in unique_nodes:
            nodes_contract=[]
            for node in G.nodes():
                if G.nodes[node]["sector"]==unique_node:#conidción donde usamos el atributo previamente definido
                       nodes_contract.append(node)
            array_nodes_contract.append(nodes_contract)
    elif len(str(arreglo[0]))==3: #caso para subsector
         for unique_node in unique_nodes:
            nodes_contract=[]
            for node in G.nodes():
                if G.nodes[node]["subsector"]==unique_node:#conidción donde usamos el atributo previamente definido
                       nodes_contract.append(node)
            array_nodes_contract.append(nodes_contract)
    elif len(str(arreglo[0]))==4: #caso para rama
         for unique_node in unique_nodes:
            nodes_contract=[]
            for node in G.nodes():
                if G.nodes[node]["rama"]==unique_node:#conidción donde usamos el atributo previamente definido
                       nodes_contract.append(node)
            array_nodes_contract.append(nodes_contract)
    if d==23:#Caso especial para recuperar los nodos 31-33 y 48-49 de la red de sector (tiene 23 nodos)
        b31_33=array_nodes_contract[4]+array_nodes_contract[5]+array_nodes_contract[6]
        b48_49=array_nodes_contract[9]+array_nodes_contract[10]
        while i<3:
            array_nodes_contract.pop(4)
            i+=1
        while j<2:
            array_nodes_contract.pop(6)
            unique_nodes.pop(5)
            j+=1
        unique_nodes.pop(8)
        array_nodes_contract.insert(4, b31_33)
        array_nodes_contract.insert(7, b48_49)     
    #print(array_nodes_contract)
    new_df=[]
    for nodes_contract1 in array_nodes_contract:
        new_file_df=[]
        nodes_contract1_asc=sorted(nodes_contract1) #ordenar los nodos en orden ascendente
        b=df2.loc[nodes_contract1_asc[0]:nodes_contract1_asc[-1], 0].sum()
        new_file_df.append(b)
        #print(nodes_contract1_asc)
        for nodes_contract2 in array_nodes_contract:
            nodes_contract2_asc=sorted(nodes_contract2) #ordenar los nodos en orden ascendente
            #print(nodes_contract2_asc)
            a=df2.loc[nodes_contract1_asc[0]:nodes_contract1_asc[-1], nodes_contract2_asc[0]:nodes_contract2_asc[-1]].sum().sum()
            #obtenemos el valor de la suma de pesos del bloque de nodos (basandonos en la matriz de adyacencia)
            new_file_df.append(a) #esta lista almacena los datos que iran en las nuevas filas del dataframe
        new_df.append(new_file_df)#almacenamos cada fila
    unique_nodes.insert(0, 0)
    #print(unique_nodes)
    df2_contract=pd.DataFrame(new_df, columns=unique_nodes, index=unique_nodes[1:])#generamos el dataframe
    if 31 in df2_contract: #caso especial sector
        df2_contract.rename(columns={31:'31-33', 48:'48-49'}, index={31:'31-33', 48:'48-49'}, inplace=True)
    return crear_red(df2_contract), df2_contract

def calculate_edge_umbral(G):
    total_edges=len(G.edges())
    if total_edges>28140: #G4-Clase
        percent=3
    elif total_edges==28140: #G3-Rama
        percent=10
    elif total_edges<=4278: #G2 and G1-Subsector y Sector
        percent=100
    decimal=percent/100
    percent_edges=int(round(total_edges*decimal))
    index=total_edges-percent_edges
    l1=[]
    for edge in G.edges():
        weight = G[edge[0]][edge[1]]['weight']
        l1.append(weight)
    l2=sorted(l1, reverse=False)
    return l2[index]

In [3]:
#Funciones clase
def create_range_a_df1_clase():
    a=list(range(3, 837)) #834 nodos
    a.insert(0, 0)
    a.insert(1, 1)
    return a

def create_df1_clase(df):
    a = create_range_a_df1_clase()
    df1=df.iloc[0:834, a] #nos quedamos con los datos de interes
    return df1

def create_dic_names_clase(df1):
    dic={}
    n=0 #contador
    for i in df1['Concepto']: #guardamos los nombres de las columnas en un diccionario
        dic[int(i[0:6])]=i[7:] #nos quedamos con los primeros 4 índices de cada categoría para asignarle un índice
        n+=1
    return dic

def create_df2_clase(df1, dic):
    a=list(range(1, 836)) #nodos+2
    #print(a)
    label_nodes=list(dic.keys())
    df2=df1.iloc[0:834, a] #ahora si tomamos la matriz de 834*835 (en este caso esta ordenada)
    df2.index=label_nodes#renombramos los índices
    label_nodes.insert(0, 0) #Signifca el UPPB
    df2.columns=label_nodes #y las columnas
    return df2

def to_pass_to_coefficients_clase(df2): #gral
    df2_copy = df2.copy() #debe ser una copia para no alterar el dataframe inicial
    for i in range(1, len(df2_copy)+1): #las 834 columnas de interes
        for j in range(0, len(df2_copy)): #834-1 (renglones)
            df2_copy.iloc[j, i]= (df2_copy.iloc[j, i])/(df2_copy.iloc[i-1, 0]) #para empezar en i=0 [row, column]
    return df2_copy

def create_network_and_df_contract_clase(df, selected_graph): #de transaction (se necesita volver a clacular cada que se cambie entre tranasacción y coeficientes)
    if selected_graph == 'G4':
        peso_arco_max_cla=red_base[1]
        return G_cla, peso_arco_max_cla, df2_cla
    elif selected_graph == 'G3':
        a_ej1=arreglo_ejemplo(G_cla, d=4)
        contraccion=contraer_red(G_cla, df2_cla, a_ej1)
        nueva_red=contraccion[0]
        G_ram=nueva_red[0]
        peso_arco_max_ram=nueva_red[1]
        df2_ram=contraccion[1]
        return G_ram, peso_arco_max_ram, df2_ram
    elif selected_graph == 'G2': #repeat
        a_ej1=arreglo_ejemplo(G_cla, d=3)
        contraccion=contraer_red(G_cla, df2_cla, a_ej1)
        nueva_red=contraccion[0]
        G_sub=nueva_red[0]
        peso_arco_max_sub=nueva_red[1]
        df2_sub=contraccion[1]
        return G_sub, peso_arco_max_sub, df2_sub
    elif selected_graph == 'G1': #repeat
        a_ej1=arreglo_ejemplo(G_cla)
        contraccion=contraer_red(G_cla, df2_cla, a_ej1)
        nueva_red=contraccion[0]
        G_sec=nueva_red[0]
        peso_arco_max_sec=nueva_red[1]
        df2_sec=contraccion[1]
        return G_sec, peso_arco_max_sec, df2_sec

def create_coefficients_network_clase(df, selected_graph):
    network_df = create_network_and_df_contract_clase(df, selected_graph)
    df2 = network_df[2]
    df2_coef = to_pass_to_coefficients_clase(df2)
    red_coef = crear_red(df2_coef)
    G_coef = red_coef[0]
    peso_arco_max_coef = red_coef[1]
    return G_coef, peso_arco_max_coef

def select_type_matrix_clase(df, type_matrix, selected_graph):
    if type_matrix == 'transaction':
        network=create_network_and_df_contract_clase(df, selected_graph)
    elif type_matrix == 'coefficients':
        network=create_coefficients_network_clase(df, selected_graph)
    return network

#Cargar red incial (solo se hace 1 vez para agilizar, por ello no se define como función)
df1=create_df1_clase(df)
dic=create_dic_names_clase(df1)
df2_cla=create_df2_clase(df1, dic)
red_base=crear_red(df2_cla)
G_cla=red_base[0]
#network=select_type_matrix_clase(df, 'transaction', 'G3')

In [4]:
#Funciones pre-gráficas
# Función para seleccionar el parámetro/métrica
def calculate_node_parameter(G, parameter, selected_parameter, selected_graph):
    if parameter == 'degree':
        dic_node_values = calculate_node_degree(G, selected_parameter)
    elif parameter == 'weight':
        dic_node_values = calculate_node_weight(G, selected_parameter)
    elif parameter == 'eigenvector centrality':
        dic_node_values = calculate_node_eigenvector_centrality(G, selected_parameter)
    elif parameter == 'betweenness centrality':
        dic_node_values = calculate_node_betweenness_centrality(G, selected_parameter)
    elif parameter == 'katz centrality':
        dic_node_values = calculate_node_katz_centrality(G, selected_parameter, selected_graph)
    elif parameter == 'pagerank':
        dic_node_values = dict(nx.pagerank(G))
    elif parameter == 'hits':
        array_dic_node_values = nx.hits(G)
        if selected_parameter == 'authorities':
            dic_node_values = array_dic_node_values[1]
        else:
            dic_node_values = array_dic_node_values[0]
    elif parameter == 'local entropy':
        dic_node_values = calculate_node_local_entropy(G, selected_parameter)
    return dic_node_values

# Función para calcular el tamaño de los nodos 
def calculate_node_local_entropy(G, selected_parameter):#in,out,total
    if selected_parameter == 'in':
        selected_degree_parameter = 'in-degree'
        selected_weight_parameter = 'in-weight'
    elif selected_parameter == 'out':
        selected_degree_parameter = 'out-degree'
        selected_weight_parameter = 'out-weight'
    elif selected_parameter == 'total':
        selected_degree_parameter = 'total-degree'
        selected_weight_parameter = 'total-weight'
    node_degrees=calculate_node_degree(G, selected_degree_parameter)
    node_weights=calculate_node_weight(G, selected_weight_parameter)
    dic_node_values={}
    for node in G.nodes():
        k_node=node_degrees[node]#obtenemos el grado
        if k_node==1 or k_node==0:#ERROR 
            entropy_node=0.0
        else:
            self_loop_weight=G.get_edge_data(node, node, 0)
            if G.has_edge(node, node) == True: #si existe un self-loop
                self_loop_weight=self_loop_weight['weight']
            F_node=node_weights[node]-self_loop_weight #F quitando el self-loop
            #if node==2379:
                #print(node_weights[node])
                #print(self_loop_weight)
                #print(F_node)
            if selected_parameter == 'in':
                edges_list=list(G.in_edges(node))
                if G.has_edge(node, node) == True: #si existe un self-loop
                    edges_list.remove((node, node))
                suma=0
                for edge in edges_list: #acá empieza la suma
                    omega_node=G[edge[0]][edge[1]]['weight']
                    suma+=(omega_node/F_node)*ma.log(omega_node/F_node)
                entropy_node=(-1/(ma.log(k_node)))*suma
            elif selected_parameter == 'out':
                edges_list=list(G.out_edges(node))
                if G.has_edge(node, node) == True: #si existe un self-loop
                    edges_list.remove((node, node))
                suma=0
                for edge in edges_list: #acá empieza la suma
                    omega_node=G[edge[0]][edge[1]]['weight']
                    delta=omega_node-F_node #tolerancia para error numérico
                    if delta>0 and delta<3.2e-13: #caso nodo 2379
                        entropy_node=0.0
                    else:
                        suma+=(omega_node/F_node)*ma.log(omega_node/F_node)
                        entropy_node=(-1/(ma.log(k_node)))*suma
            elif selected_parameter == 'total':
                l1=list(G.in_edges(node))
                l2=list(G.out_edges(node))
                edges_list=list(set(l1+l2))#total edges
                if G.has_edge(node, node) == True: #si existe un self-loop
                    edges_list.remove((node, node))
                suma=0
                for edge in edges_list: #acá empieza la suma
                    omega_node=G[edge[0]][edge[1]]['weight']
                    suma+=(omega_node/F_node)*ma.log(omega_node/F_node)
                entropy_node=(-1/(ma.log(k_node)))*suma
        dic_node_values[node]=entropy_node
    return dic_node_values               

# Función para calcular el tamaño de los nodos 
def calculate_node_eigenvector_centrality(G, selected_parameter):
    if selected_parameter == 'weight':
        dic_node_values = dict(nx.eigenvector_centrality(G, weight='weight'))
    else:
        dic_node_values = dict(nx.eigenvector_centrality(G))
    return dic_node_values

# Función para calcular el tamaño de los nodos 
def calculate_node_betweenness_centrality(G, selected_parameter):
    if selected_parameter == 'weight':
        dic_node_values = dict(nx.betweenness_centrality(G, weight='weight'))
    else:
        dic_node_values = dict(nx.betweenness_centrality(G))
    return dic_node_values

# Función para calcular el tamaño de los nodos 
def calculate_node_katz_centrality(G, selected_parameter, selected_graph):#meter condiciones sobre el tipo de matriz
    if selected_parameter == 'weight':
        if selected_graph=='G1':
            alpha_value=0.0000027 #el tipo de matriz cambia este valor
        elif selected_graph=='G2':
            alpha_value=0.000000392
        elif selected_graph=='G3':
            alpha_value=3 #0.00000392
        elif selected_graph=='G4':
            alpha_value=1.5
        dic_node_values = dict(nx.katz_centrality(G, weight='weight', alpha=alpha_value))
    else:
        if selected_graph=='G2':
            alpha_value=0.0174
        elif selected_graph=='G1':
            alpha_value=0.0533
        elif selected_graph=='G3':
            alpha_value=0.00692
        elif selected_graph=='G4':
            alpha_value=0.00392
        dic_node_values = dict(nx.katz_centrality(G, alpha=alpha_value))
    return dic_node_values

# Función para calcular el tamaño de los nodos según el tipo de peso seleccionado y el grafo
def calculate_node_weight(G, selected_weight):
    if selected_weight == 'in-weight':
        node_weights = dict(G.in_degree(weight='weight'))
    elif selected_weight == 'out-weight':
        node_weights = dict(G.out_degree(weight='weight'))
    else:
        node_weights = dict(G.degree(weight='weight'))
    return node_weights

# Función para calcular el tamaño de los nodos según el tipo de grado seleccionado y el grafo
def calculate_node_degree(G, selected_degree):
    if selected_degree == 'in-degree':
        node_degrees = dict(G.in_degree)
    elif selected_degree == 'out-degree':
        node_degrees = dict(G.out_degree)
    else:
        node_degrees = dict(G.degree)
    return node_degrees

# Modificar el grosor de las aristas según su peso y el grafo
def new_edges(G, peso_arco_max):
    umbral_pesos=calculate_edge_umbral(G)
    for edge in G.edges():
        factor_de_normalizacion_pesos= n_pesos/peso_arco_max
        source, target = str(edge[0]), str(edge[1])
        weight = G[edge[0]][edge[1]]['weight']
        if weight>umbral_pesos: #total weight
            stylesheet.append({'selector': '[source = "{}"][target = "{}"]'.format(source, target),
                               'style': {'width': weight*factor_de_normalizacion_pesos}}) #opacity
    return stylesheet

# Función para asignar el tamaño del nodo según el tipo de parámetro seleccionado y el grafo
# Además asigna el color dependiendo del tipo de grado seleccionado
def new_node_stylesheet(G, parameter, selected_parameter, parameter1, selected_parameter1, n_size, selected_graph):
    dic_node_values = calculate_node_parameter(G, parameter, selected_parameter, selected_graph)
    dic_node_colors = calculate_node_parameter(G, parameter1, selected_parameter1, selected_graph)
    normalizar_size = max(list(dic_node_values.values())) 
    factor_de_normalizacion_values=n_size/normalizar_size
    normalizar_color = max(list(dic_node_colors.values())) #los valores para el color son entre 0 y 1
    #factor_de_normalizacion_colors=n_size/normalizar_color
    cmap = cm.viridis #mapa de colores
    bins =  np.linspace(0, 1, n_bins) #segmentos de color
    val = {}
    for node_id, value in dic_node_colors.items():
        val[node_id]= (value/(normalizar_color)) #valor de color normalizado
    cero_escala_color=min(list(val.values())) #hace que el color inicial pertenezca al valor mínimo
    for node_id, value in dic_node_values.items():
        idx, hexc = val_to_color(val[node_id], np.linspace(cero_escala_color, 1, n_bins), cmap_at_scale(cmap, bins))
        stylesheet.append({'selector': '[id = "{}"]'.format(node_id), 
                           'style': {'width': value*factor_de_normalizacion_values,  'font-size': '10px',      # establece el tamaño de la fuente de la etiqueta 
                                     'height': value*factor_de_normalizacion_values, 'background-color': hexc}})
    return stylesheet

def update_options(parameter):
    if parameter == 'degree':
        return [{'label': 'In-degree', 'value': 'in-degree'},{'label': 'Out-degree', 'value': 'out-degree'},
                {'label': 'Total degree', 'value': 'total'}]
    elif parameter == 'weight':
        return [{'label': 'In-weight', 'value': 'in-weight'},{'label': 'Out-weight', 'value': 'out-weight'},
                {'label': 'Total weight', 'value': 'total'}]
    elif parameter == 'katz centrality':
        return [{'label': 'With weight', 'value': 'total'}]
    elif parameter == 'pagerank':
        return [{'label': 'None', 'value': 'total'}]
    elif parameter == 'hits':
        return [{'label': 'Hubs', 'value': 'total'}, {'label': 'Authorities', 'value': 'authorities'}]
    elif parameter == 'local entropy':
        return [{'label': 'In', 'value': 'in'}, {'label': 'Out', 'value': 'out'}, {'label': 'Total', 'value': 'total'}]
    else:
        return [{'label': 'Without weight', 'value': 'total'}, {'label': 'With weight', 'value': 'weight'}]
    return []

#Función de la barra de color
def create_color_bar(G, parameter, selected_parameter, selected_graph):
    # Crear el gráfico con Matplotlib
    fig, ax = plt.subplots(figsize=(8, 0.25))
    dic_node_values = calculate_node_parameter(G, parameter, selected_parameter, selected_graph)
    parameter_type_options=update_options(parameter)
    for dic in parameter_type_options:
        if dic['value'] == selected_parameter:
            label_selected_parameter=dic['label']
    cmap = cm.viridis #mapa de colores
    norm = mpl.colors.Normalize(vmin=min(dic_node_values.values()), vmax=max(dic_node_values.values()))
    cbar=fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=ax, orientation='horizontal', label=f'Node {parameter} ({label_selected_parameter})')
    # Convertir el gráfico de Matplotlib a una imagen
    img = io.BytesIO()
    plt.savefig(img, format='png', bbox_inches='tight')
    img.seek(0)
    img_base64 = base64.b64encode(img.getvalue()).decode('utf-8')
    # Ocultar la visualización del gráfico de Matplotlib
    plt.close()
    return img_base64

# Crear elementos del grafo (nodos y arcos)
def create_elements(G):
    umbral_pesos=calculate_edge_umbral(G)
    elements = []
    for edge in G.edges():
        weight = G[edge[0]][edge[1]]['weight']
        if weight>umbral_pesos:
            elements.append({'data': {'source': str(edge[0]), 'target': str(edge[1]), 'weight':G[edge[0]][edge[1]]['weight']}})
    for node in G.nodes():
        elements.append({'data': {'id': str(node)}})
    return elements

#Función para la lista del top nodos
def calcula_top_nodos(G, parameter, selected_parameter, selected_graph, n=10):
    dic_top={}
    dic_node_values=calculate_node_parameter(G, parameter, selected_parameter, selected_graph)
    values=list(dic_node_values.values())
    keys=list(dic_node_values.keys())
    list_desc_nodes_values=sorted(values, reverse=True)
    top_nodes_values=list_desc_nodes_values[0:n]
    for node_value_top in top_nodes_values:
        i=0 #contador           
        node_value=values[i]
        while node_value_top!=node_value:        
            i+=1
            node_value=values[i]        
        dic_top[keys[values.index(node_value)]]=node_value
        values[values.index(node_value)]=-1 #para no repetir nodos
    #ahora a imprimir la lista de nodos en formato    
    markdown_lista=f'''### Top nodes - Value of {parameter}
    
'''
    j=1
    for x, y in dic_top.items():
        if parameter=='degree':
            markdown_lista+=f'''{j}. {x} - {y}
'''
            j+=1
        elif parameter=='weight' or parameter=='eigenvector centrality':
            markdown_lista+=f'''{j}. {x} - {round(y, 2)}
'''
            j+=1
        else:
            markdown_lista+=f'''{j}. {x} - {round(y, 7)}
'''
            j+=1
    return markdown_lista

def graphing_distribution(G, parameter, selected_parameter, selected_graph):
    dic_node_values=calculate_node_parameter(G, parameter, selected_parameter, selected_graph)
    data = list(dic_node_values.values())
    fig = px.histogram(data, width=350, height=400)
    fig.update_layout(title=f"Distribution of size parameter type", xaxis_title="Value",
                      yaxis_title="Count", font=dict(size=9, color="Black"), showlegend=False,
                     margin=dict(l=10, r=5, t=60, b=5))
    return fig

In [5]:
#Gráfico cytoscape-dash-callbacks
cyto.load_extra_layouts()

#Parámetros iniciales
n_pesos=3 #factor para normalizar el valor del grosor de las líneas
n_bins=15 #número de divisiones de la escala de color 
GRAFO_SELECCIONADO=select_type_matrix_clase(df, 'transaction', 'G4')[0]#clase

# Lista de estilos
stylesheet = [{'selector': 'node', 'style': {'label': 'data(id)', 'shape': 'ellipse', 'opacity': 0.8,
                                             'border-color': 'black', 'border-width': 1, 'border-opacity': 0.8}},
              {'selector': 'edge', 'style': { 'opacity': 0.7, 'curve-style': 'straight', 'target-arrow-shape': 'vee',
                                             'target-arrow-opacity': 0.6}}]

# Crear la aplicación Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2('MIP 19: Producto por producto, origen doméstico (2018)'),
    html.Div([
        html.Div([
           html.Label('Matrix:'),
           dcc.Dropdown(id='matrix-dropdown', options=[{'label': 'Transaction', 'value': 'transaction'},
                                               {'label': 'Technical Coefficients', 'value': 'coefficients'}], value='transaction')
        ], 
        style={'width': '14%', 'display': 'inline-block'}),
        html.Div([
           html.Label('Graph:'),
           dcc.Dropdown(id='graph-dropdown', options=[{'label': 'Sector', 'value': 'G1'},
                                                      {'label': 'Subsector', 'value': 'G2'},
                                                      {'label': 'Rama', 'value': 'G3'},
                                                      {'label': 'Class', 'value': 'G4'}], value='G4')
        ], 
        style={'width': '12%', 'display': 'inline-block'}),
        html.Div([
           html.Label('Size parameter:'),
           dcc.Dropdown(id='parameter-dropdown', options=[{'label': 'Degree', 'value': 'degree'},
                                               {'label': 'Weight', 'value': 'weight'},
                                                {'label': 'Eigenvector Centrality', 'value': 'eigenvector centrality'},
                                                {'label': 'Betweenness Centrality', 'value': 'betweenness centrality'},
                                                {'label': 'Katz Centrality', 'value': 'katz centrality'},
                                                {'label': 'Pagerank', 'value': 'pagerank'},
                                                {'label': 'Hits', 'value': 'hits'},
                                                {'label': 'Local entropy', 'value': 'local entropy'}],
                        value='weight')
        ], 
        style={'width': '14%', 'display': 'inline-block'}),
        html.Div([
            html.Label('Size parameter type:'),
            dcc.Dropdown(id='parameter-type-dropdown', value='total')
        ], 
        style={'width': '16%', 'display': 'inline-block'}),
        html.Div([
            html.Label('Layout type:'),
            dcc.Dropdown(id='dropdown-update-layout', value='preset', clearable=False, options=[
                {'label': name.capitalize(), 'value': name} for name in ['preset','spring1','spring2', 'kamada-kawai',
                                                                         'spectral', 'grid', 'circle', 'random',
                                                                         'concentric', 'breadthfirst', 
                                                                         'F-R', 'cose-bilkent']])
        ], 
        style={'width': '14%', 'display': 'inline-block'}),
        html.Div([
            html.Label('Color parameter:'),
            dcc.Dropdown(id='color-parameter-dropdown', options=[{'label': 'Degree', 'value': 'degree'},
                                               {'label': 'Weight', 'value': 'weight'},
                                                {'label': 'Eigenvector Centrality', 'value': 'eigenvector centrality'},
                                                {'label': 'Betweenness Centrality', 'value': 'betweenness centrality'},
                                                {'label': 'Katz Centrality', 'value': 'katz centrality'},
                                                {'label': 'Pagerank', 'value': 'pagerank'},
                                                {'label': 'Hits', 'value': 'hits'},
                                                {'label': 'Local entropy', 'value': 'local entropy'}],
                        value='degree')#usamos por default el grado para el color
        ], 
        style={'width': '14%', 'display': 'inline-block'}),
        html.Div([
            html.Label('Color parameter type:'),
            dcc.Dropdown(id='color-parameter-type-dropdown', value='total')
        ], 
        style={'width': '15%', 'display': 'inline-block'})
    ]),
    html.Div([
        html.Div([
            dcc.Markdown(id='list-top-nodes', children=calcula_top_nodos(GRAFO_SELECCIONADO, 'weight', 'total', 'G4'))
        ],
        style={'width': '15%', #'display': 'inline-block', 
               'position': 'absolute', 
               'top': '140px',  # Valor para mover el markdown verticalmente
               'left': '0px',  # Valor para mover el markdown horizontalmente
               'background-color': 'rgba(255, 255, 255, 0.7)',  # Fondo semitransparente
               'padding': '10px',
               'border-radius': '5px',
               'z-index': '10',  # Asegura que el markdown se vea por encima del gráfico
               'font-size': '10px' }),
        html.Div([
            cyto.Cytoscape(id='network-graph', layout={'name': 'preset'}, style={'width': '100%', 'height': '370px', 'marginTop': '5px'},
                   elements=create_elements(GRAFO_SELECCIONADO),  # Agregar elementos al grafo
                   stylesheet=[{'selector': 'node', 'style': {'label': 'data(id)', 'shape': 'ellipse', 'opacity': 0.8,
                                                              'border-color': 'black', 'border-width': 1,
                                                              'border-opacity': 0.8}},
                               {'selector': 'edge', 'style': {'opacity': 0.7,'curve-style': 'straight',
                                                              #'width': 1, #default
                                                              'target-arrow-shape': 'vee',  # Establece la forma de la flecha
                                                              #'target-arrow-color': '#000', # Establece el color de la flecha
                                                              'target-arrow-opacity': 0.6}}])
        ],
        style={'width': '64%', 'height': '370px', 'display': 'inline-block'}),
        html.Div([
            dcc.Graph(id='distribution-graph', figure=graphing_distribution(GRAFO_SELECCIONADO, 'weight', 'total', 'G4'))
        ],
        style={'width': '36%', 'height': '370px', 'display': 'inline-block'})
    ]),
    html.Div([
        html.Img(id='colorbar', src='', width='auto', height='auto')
    ])
])


# Callback para actualizar la distribución dependiendo del paramter and type-parameter-dropdown
@app.callback(Output('distribution-graph', 'figure'), [Input('graph-dropdown', 'value'),
                                                      Input('matrix-dropdown', 'value'), 
                                                      Input('parameter-dropdown', 'value'),
                                                      Input('parameter-type-dropdown', 'value')])
def update_distribution_graph(selected_graph, type_matrix, parameter, selected_parameter):
    network_and_arc=select_type_matrix_clase(df, type_matrix, selected_graph)
    G = network_and_arc[0]
    return graphing_distribution(G, parameter, selected_parameter, selected_graph)

# Callback para actualizar las opciones del type-parameter-dropdown dependiendo del parametro
@app.callback(Output('parameter-type-dropdown', 'options'), [Input('parameter-dropdown', 'value')])
def update_parameter_type_options(parameter):
    return update_options(parameter)

# Callback para actualizar las opciones del color-type-parameter-dropdown dependiendo del parametro
@app.callback(Output('color-parameter-type-dropdown', 'options'), [Input('color-parameter-dropdown', 'value')])
def update_parameter_type_color_options(parameter):
    return update_options(parameter)

# Callback para actualizar el grafo seleccionado (actualiza los respectivos bordes)
@app.callback(Output('network-graph', 'elements'), [Input('graph-dropdown', 'value'),
                                                    Input('matrix-dropdown', 'value')])
def update_graph_elements(selected_graph, type_matrix):
    network_and_arc=select_type_matrix_clase(df, type_matrix, selected_graph)
    G, peso_arco_max = network_and_arc[0], network_and_arc[1]
    new_edges(G, peso_arco_max)    
    return create_elements(G)

# Callback para actualizar el tamaño de los nodos del grafo cuando cambia el tipo y parametro seleccionado
@app.callback(Output('network-graph', 'stylesheet'), [Input('graph-dropdown', 'value'),
                                                      Input('matrix-dropdown', 'value'),
                                                      Input('parameter-dropdown', 'value'),
                                                      Input('parameter-type-dropdown', 'value'),
                                                      Input('color-parameter-dropdown', 'value'),
                                                      Input('color-parameter-type-dropdown', 'value')])
def update_stylesheet(selected_graph, type_matrix, parameter, selected_parameter, parameter1, selected_parameter1):
    network_and_arc=select_type_matrix_clase(df, type_matrix, selected_graph)
    G = network_and_arc[0]
    if parameter=='degree':
        n_size=25 #factor para normalizar el valor del tamaño del nodo
    elif parameter=='weight':
        if selected_graph=='G1':
            n_size=100
        else:
            n_size=40
    elif parameter=='eigenvector centrality':
        n_size=100 #25 sin weight
    else:
        n_size=25
    new_stylesheet = new_node_stylesheet(G, parameter, selected_parameter, parameter1, selected_parameter1, n_size, selected_graph)
    return stylesheet

# Callback para actualizar el colorbar
@app.callback(Output('colorbar', 'src'), [Input('graph-dropdown', 'value'),
                                          Input('matrix-dropdown', 'value'),
                                          Input('color-parameter-dropdown', 'value'),
                                          Input('color-parameter-type-dropdown', 'value')])
def update_colorbar(selected_graph, type_matrix, parameter, selected_parameter):
    network_and_arc=select_type_matrix_clase(df, type_matrix, selected_graph)
    G = network_and_arc[0]
    img_base64 = create_color_bar(G, parameter, selected_parameter, selected_graph)
    return 'data:image/png;base64,{}'.format(img_base64)

# Callback para actualizar la lista de nodos
@app.callback(Output('list-top-nodes', 'children'), [Input('graph-dropdown', 'value'),
                                                     Input('matrix-dropdown', 'value'),
                                                     Input('parameter-dropdown', 'value'),
                                                     Input('parameter-type-dropdown', 'value')])
def update_list_top_nodes(selected_graph, type_matrix, parameter, selected_parameter):
    network_and_arc=select_type_matrix_clase(df, type_matrix, selected_graph)
    G = network_and_arc[0]
    new_markdown_list=calcula_top_nodos(G, parameter, selected_parameter, selected_graph)
    return new_markdown_list

# Callback para cambiar el layout
@app.callback(Output('network-graph', 'layout'), [Input('graph-dropdown', 'value'),
                                                  Input('matrix-dropdown', 'value'),
                                                  Input('parameter-dropdown', 'value'),
                                                  Input('dropdown-update-layout', 'value')])
def update_layout(selected_graph, type_matrix, parameter, layout):
    network_and_arc=select_type_matrix_clase(df, type_matrix, selected_graph)
    G = network_and_arc[0]
    if parameter=='degree':
        if selected_graph=='G1':
            n_pos=250 #factor para controlar el espaciado
        else:
            n_pos=900
    elif parameter=='weight':
        if selected_graph=='G1':
            n_pos=225 #factor para controlar el espaciado
        else:
            n_pos=700
    elif parameter=='eigenvector centrality':
        if selected_graph=='G1':
            n_pos=300
        else:
            n_pos=600
    elif parameter=='betweenness centrality':
        if selected_graph=='G1':
            n_pos=300
        else:
            n_pos=600
    elif parameter=='katz centrality':
        if selected_graph=='G1':
            n_pos=300
        else:
            n_pos=600
    elif parameter=='pagerank':
        if selected_graph=='G1':
            n_pos=300
        else:
            n_pos=600
    elif parameter=='hits':###
        if selected_graph=='G1':
            n_pos=300
        else:
            n_pos=600
    elif parameter=='local entropy':###
        if selected_graph=='G1':
            n_pos=300
        else:
            n_pos=600
    
    if layout=='spring1' or layout=='spring2' or layout=='kamada-kawai' or layout=='spectral':
        if (layout=='spring1') or (layout=='spring2'):
            if layout=='spring1' and selected_graph=='G1' and parameter=='degree':
                k=565
            elif layout=='spring1' and selected_graph=='G2' and parameter=='degree':
                k=205
            elif layout=='spring1' and selected_graph=='G3' and parameter=='degree':#
                k=205
            elif layout=='spring1' and selected_graph=='G4' and parameter=='degree':#
                k=205
            elif layout=='spring2' and selected_graph=='G1' and parameter=='degree':
                k=0.01
            elif layout=='spring2' and selected_graph=='G2' and parameter=='degree':
                k=145
            elif layout=='spring2' and selected_graph=='G3' and parameter=='degree':#
                k=145
            elif layout=='spring2' and selected_graph=='G4' and parameter=='degree':#
                k=145
            elif layout=='spring1' and selected_graph=='G1' and parameter=='weight':
                k=650
            elif layout=='spring1' and selected_graph=='G2' and parameter=='weight':
                k=300
            elif layout=='spring1' and selected_graph=='G3' and parameter=='weight':#
                k=300
            elif layout=='spring1' and selected_graph=='G4' and parameter=='weight':#
                k=300
            elif layout=='spring2' and selected_graph=='G1' and parameter=='weight':
                k=0.01
            elif layout=='spring2' and selected_graph=='G2' and parameter=='weight':
                k=185
            elif layout=='spring2' and selected_graph=='G3' and parameter=='weight':#
                k=185
            elif layout=='spring2' and selected_graph=='G4' and parameter=='weight':#
                k=185
            elif layout=='spring1' and selected_graph=='G1' and parameter=='eigenvector centrality':
                k=900 #700
            elif layout=='spring1' and selected_graph=='G2' and parameter=='eigenvector centrality':
                k=310 #225
            elif layout=='spring1' and selected_graph=='G3' and parameter=='eigenvector centrality':#
                k=310 #225
            elif layout=='spring1' and selected_graph=='G4' and parameter=='eigenvector centrality':#
                k=310 #225
            elif layout=='spring2' and selected_graph=='G1' and parameter=='eigenvector centrality':
                k=1000 #960
            elif layout=='spring2' and selected_graph=='G2' and parameter=='eigenvector centrality':
                k=325 #250
            elif layout=='spring2' and selected_graph=='G3' and parameter=='eigenvector centrality':#
                k=325 #250
            elif layout=='spring2' and selected_graph=='G4' and parameter=='eigenvector centrality':#
                k=325 #250
            elif layout=='spring1' and selected_graph=='G1' and parameter=='betweenness centrality':
                k=900 #700
            elif layout=='spring1' and selected_graph=='G2' and parameter=='betweenness centrality':
                k=350 #225
            elif layout=='spring1' and selected_graph=='G3' and parameter=='betweenness centrality':#
                k=350 #225
            elif layout=='spring1' and selected_graph=='G4' and parameter=='betweenness centrality':#
                k=350 #225
            elif layout=='spring2' and selected_graph=='G1' and parameter=='betweenness centrality':
                k=1000 #960
            elif layout=='spring2' and selected_graph=='G2' and parameter=='betweenness centrality':
                k=325 #250
            elif layout=='spring2' and selected_graph=='G3' and parameter=='betweenness centrality':#
                k=325 #250
            elif layout=='spring2' and selected_graph=='G4' and parameter=='betweenness centrality':#
                k=325 #250
            elif layout=='spring1' and selected_graph=='G1' and parameter=='katz centrality':
                k=900 #750
            elif layout=='spring1' and selected_graph=='G2' and parameter=='katz centrality':
                k=350 #225
            elif layout=='spring1' and selected_graph=='G3' and parameter=='katz centrality':#
                k=350 #225
            elif layout=='spring1' and selected_graph=='G4' and parameter=='katz centrality':#
                k=350 #225
            elif layout=='spring2' and selected_graph=='G1' and parameter=='katz centrality':
                k=1000 #960
            elif layout=='spring2' and selected_graph=='G2' and parameter=='katz centrality':
                k=325 #250
            elif layout=='spring2' and selected_graph=='G3' and parameter=='katz centrality':#
                k=325 #250
            elif layout=='spring2' and selected_graph=='G4' and parameter=='katz centrality':#
                k=325 #250
            elif layout=='spring1' and selected_graph=='G1' and parameter=='pagerank':
                k=900 #700
            elif layout=='spring1' and selected_graph=='G2' and parameter=='pagerank':
                k=350 #225
            elif layout=='spring1' and selected_graph=='G3' and parameter=='pagerank':#
                k=350 #225
            elif layout=='spring1' and selected_graph=='G4' and parameter=='pagerank':#
                k=350 #225
            elif layout=='spring2' and selected_graph=='G1' and parameter=='pagerank':
                k=1000 #960
            elif layout=='spring2' and selected_graph=='G2' and parameter=='pagerank':
                k=325 #250
            elif layout=='spring2' and selected_graph=='G3' and parameter=='pagerank':#
                k=325 #250
            elif layout=='spring2' and selected_graph=='G4' and parameter=='pagerank':#
                k=325 #250
            elif layout=='spring1' and selected_graph=='G1' and parameter=='hits':
                k=900 #750
            elif layout=='spring1' and selected_graph=='G2' and parameter=='hits':
                k=350 #225
            elif layout=='spring1' and selected_graph=='G3' and parameter=='hits':#
                k=350 #225
            elif layout=='spring1' and selected_graph=='G4' and parameter=='hits':#
                k=350 #225
            elif layout=='spring2' and selected_graph=='G1' and parameter=='hits':
                k=1000 #960
            elif layout=='spring2' and selected_graph=='G2' and parameter=='hits':
                k=325 #250
            elif layout=='spring2' and selected_graph=='G3' and parameter=='hits':#
                k=325 #250
            elif layout=='spring2' and selected_graph=='G4' and parameter=='hits':#
                k=325 #225
            elif layout=='spring1' and selected_graph=='G1' and parameter=='local entropy':
                k=900 #750
            elif layout=='spring1' and selected_graph=='G2' and parameter=='local entropy':
                k=350 #225
            elif layout=='spring1' and selected_graph=='G3' and parameter=='local entropy':#
                k=350 #225
            elif layout=='spring1' and selected_graph=='G4' and parameter=='local entropy':#
                k=350 #225
            elif layout=='spring2' and selected_graph=='G1' and parameter=='local entropy':
                k=1000 #960
            elif layout=='spring2' and selected_graph=='G2' and parameter=='local entropy':
                k=325 #250
            elif layout=='spring2' and selected_graph=='G3' and parameter=='local entropy':#
                k=325 #250
            elif layout=='spring2' and selected_graph=='G4' and parameter=='local entropy':#
                k=325 #225
            pos=nx.spring_layout(G, k, iterations=1000, scale=n_pos)
        elif layout=='kamada-kawai': 
            if selected_graph=='G1' and (parameter=='degree' or parameter=='weight'):#
                scale_number=500
            elif selected_graph=='G2' and (parameter=='degree' or parameter=='weight'):#
                scale_number=120000
            elif selected_graph=='G3' and (parameter=='degree' or parameter=='weight'):#
                scale_number=120000
            elif selected_graph=='G4' and (parameter=='degree' or parameter=='weight'):#
                scale_number=500
            pos=nx.kamada_kawai_layout(G, weight='weight', scale=scale_number)
        elif layout=='spectral':
            if selected_graph=='G1' and (parameter=='degree' or parameter=='weight'):#
                scale_number=1
            elif selected_graph=='G2' and (parameter=='degree' or parameter=='weight'):#
                scale_number=5000
            pos=nx.spectral_layout(G, weight='weight')#, scale=scale_number)
        dic_node={}
        for node in pos:
            #if  layout=='kamada-kawai' and node==211:
            dic_node[node]={'x':pos[node][0], 'y':pos[node][1]}
        return {'name': 'preset', 'positions': dic_node, 'padding': 1, 'animate': True}
    
    if layout== 'F-R' or layout=='cose-bilkent':
        dic={'randomize':True, 'fit':True, 'padding':1, 'animate':True, 'nodeDimensionsIncludeLabels': False,
             'numIter':1500}
        atrib=['name', 'nodeRepulsion', 'idealEdgeLength', 'edgeElasticity', 'gravity', 'refresh']
        if parameter=='degree':            
            a_fr_g1=['cose', 4000000, 100, 45, 1, 3]
            a_fr_g2=['cose', 6000000, 150, 75, 2, 30]            
            a_cb_g1=[layout, 8000, 125, 70, 0.5, 3]
            a_cb_g2=[layout, 800000, 200, 0.45, 1, 100]
        elif parameter=='weight':            
            a_fr_g1=['cose', 1300000, 200, 45, 2, 3]
            a_fr_g2=['cose', 4000000, 150, 100, 80, 20]            
            a_cb_g1=[layout, 6000, 100, 0.45, 1, 30]
            a_cb_g2=[layout, 450000, 100, 0.45, 1, 30]
        i, j=0, 0
        if layout=='F-R':
            atrib_fr=['componentSpacing', 'nodeOverlap', 'initialTemp', 'coolingFactor', 'minTemp']
            if parameter=='degree':                
                b_fr_g1=[100, 10, 200, 0.97, 1.0]
                b_fr_g2=[100, 25, 200, 0.95, 1.0]
            elif parameter=='weight':                
                b_fr_g1=[100, 12, 200, 0.95, 1.0]
                b_fr_g2=[100, 10, 200, 0.97, 1.0]
            if selected_graph=='G1':
                for a in atrib:
                    dic[a]=a_fr_g1[i]
                    i+=1
                for b in atrib_fr:
                    dic[b]=b_fr_g1[j]
                    j+=1
            elif selected_graph=='G2':
                for a in atrib:
                    dic[a]=a_fr_g2[i]
                    i+=1
                for b in atrib_fr:
                    dic[b]=b_fr_g2[j]
                    j+=1
        elif layout=='cose-bilkent':
            dic1={'tile':True, 'tilingPaddingVertical':50, 'tilingPaddingHorizontal':50, 'quality':'default'}
            dic.update(dic1)
            atrib_cb=['gravityRange', 'initialEnergyOnIncremental']
            if parameter=='degree':
                b_cb_g1=[3.8, 4]
                b_cb_g2=[3.8, 3]
            elif parameter=='weight':
                b_cb_g1=[3.8, 3]
                b_cb_g2=[3.8, 5]
            if selected_graph=='G1':
                for a in atrib:
                    dic[a]=a_cb_g1[i]
                    i+=1
                for b in atrib_cb:
                    dic[b]=b_cb_g1[j]
                    j+=1
            elif selected_graph=='G2':
                for a in atrib:
                    dic[a]=a_cb_g2[i]
                    i+=1
                for b in atrib_cb:
                    dic[b]=b_cb_g2[j]
                    j+=1
        return dic
    else:
        return {'name': layout, 'animate': True}

if __name__ == '__main__':
    app.run_server(debug=True)
    #app.run_server(port=2223) #error port